# Prediction with TropiSEQ :
### I- Prepare the model
### II- Run the predictions on matrices
### III- Run the predictions on experimentally validated depolymerases¶
***

In [2]:
# Ground modules
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SeqIO
from itertools import product
import random
from collections import Counter
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import logging
import subprocess
from multiprocessing.pool import ThreadPool
import joblib
import json

# SCikitlearn modules :
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report , roc_auc_score

# Scipy modules : 
from scipy.stats import fisher_exact

> Set up predictor 

In [3]:
path_seqbased = "/media/concha-eloko/Linux/PPT_clean/Seqbased_model"
path_db = f"{path_seqbased}/TropiSeq/TropiLR_0.65.db"
path_work = "/media/concha-eloko/Linux/PPT_clean"
path_benchmark = "/media/concha-eloko/Linux/PPT_clean/benchmark"

# Run makeblast command :
fasta_file = f"{path_seqbased}/cdhit_clusters_2912/0.65.out"

blast_command = f"makeblastdb -in {fasta_file} -dbtype prot -out {path_db}"
#make_blast_process = subprocess.Popen(blast_command, shell =True, stdout = subprocess.PIPE, stderr=subprocess.STDOUT)
#mkblast_out, mkblast_err = make_blast_process.communicate()
#print(mkblast_out , mkblast_err)

# Relevant files :
dico_cluster = json.load(open(f"{path_seqbased}/dico_cluster.cdhit__0.65.json"))
dico_cluster_r = {ref_dpo : key_dpo for key_dpo,list_dpo in dico_cluster.items() for ref_dpo in list_dpo}

b'\n\nBuilding a new DB, current time: 12/05/2024 15:56:05\nNew DB name:   /media/concha-eloko/Linux/PPT_clean/Seqbased_model/TropiSeq/TropiLR_0.65.db\nNew DB title:  /media/concha-eloko/Linux/PPT_clean/Seqbased_model/cdhit_clusters_2912/0.65.out\nSequence type: Protein\nKeep MBits: T\nMaximum file size: 1000000000B\nAdding sequences from FASTA; added 836 sequences in 0.172458 seconds.\n' None


In [4]:
# Relevant files :
dico_cluster = json.load(open(f"{path_seqbased}/dico_cluster.cdhit__0.65.json"))
dico_cluster_r = {ref_dpo : key_dpo for key_dpo,list_dpo in dico_cluster.items() for ref_dpo in list_dpo}

> Load predictor

In [14]:
final_annotation = pd.read_csv(f"/media/concha-eloko/Linux/PPT_clean/Seqbased_model/labeling_depo_clusters.pred.LogReg.tsv", sep = "\t", header = 0)

final_annotation_tropiseq = final_annotation[final_annotation["TropiLR_KL_types"] != "None"]
final_annotation_tropiseq = final_annotation_tropiseq.drop_duplicates(subset = ["depo_cluster", "TropiLR_KL_types"])
final_annotation_tropiseq = final_annotation_tropiseq[["depo_cluster", "TropiLR_KL_types", "TropiLR_scores"]]

dico_tropiseq_data = {row["depo_cluster"] : {"KL_types" : row["TropiLR_KL_types"], "Scores" : row["TropiLR_scores"]} for _, row in final_annotation_tropiseq.iterrows()}


In [16]:
dico_pred = json.load(open("/media/concha-eloko/Linux/PPT_clean/Seqbased_model/prediction_based.labeling.LogReg.json"))
dico_pred_correct_name = {f"Dpo_cdhit_{cluster.split('_')[1]}":hits  for cluster, hits in dico_pred.items()}


***
### Run predictions on matrices: 

In [17]:
path_tmp =  "/media/concha-eloko/Linux/PPT_clean/Seqbased_model/tmp"
labels_blast=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]

def tmp_fasta_file(record , path_tmp) :
    name_file = "_".join(record.description.split(" "))
    path_fasta = f"{path_tmp}/{name_file}.fasta"
    length_seq = len(record.seq)
    with open(path_fasta, "w") as outfile :
        outfile.write(f">{record.description}\n{str(record.seq)}")
    return path_fasta , length_seq

def blast_seq(path_fasta, path_DB, path_tmp) :
    file_name = path_fasta.split("/")[-1]
    command = f"blastp -query {path_fasta} -db {path_DB} -out {path_tmp}/{file_name}.blast_out -outfmt 6 -evalue 1e-10"
    blastp_sub = subprocess.Popen(command ,shell = True, stdout = subprocess.PIPE, stderr = subprocess.STDOUT)
    out , err = blastp_sub.communicate()
    return f"{path_tmp}/{file_name}.blast_out"

def get_best_candidate(path_blast_out, length_seq, bitscore = 75) : 
    winner = 0
    labels_blast=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]
    blast_df = pd.read_csv(path_blast_out, sep = "\t", names = labels_blast)
    if len(blast_df) > 0 :
        row = blast_df.iloc[0] 
        if row["bitscore"] > bitscore and length_seq/int(row["length"])> 0.8:
            winner = dico_cluster_r[row["sseqid"]]
        else :
            winner = "No hits"
    else :
        winner = "No hits"
    return winner

def get_winner(record , path_tmp) :
    path_func , len_func = tmp_fasta_file(record, path_tmp)
    path_blast_out_func = blast_seq(path_func , path_db, path_tmp)
    winner = get_best_candidate(path_blast_out_func, len_func)
    return winner

In [18]:
# Ferriol inferences : 
path_seq = "/media/concha-eloko/Linux/77_strains_phage_project/rbp_work"
dico_seq = {record.description : record.seq for record in SeqIO.parse(f"{path_seq}/77_phages_Dpo_domains.2406.multi.fasta", "fasta") if len(record.seq) >0}
set_records = [record for record in SeqIO.parse(f"{path_seq}/77_phages_Dpo_domains.2406.multi.fasta", "fasta") if len(record.seq) > 0]


ferriol_winners = []
for record in tqdm(set_records) :
    winner = get_winner(record, path_tmp)
    if winner != "No hits" :
        hit = int(winner.split("_")[-1])
        results = dico_pred_correct_name.get(winner, {})
        a = (record.description.split(",")[0] , winner, results)
    else :
        results = "Null"
    a = (record.description.split(",")[0] , winner, results)
    ferriol_winners.append(a)
    

# ***************************************************************************
# Beamud inferences : 
bea_winners = []
path_bea = "/media/concha-eloko/Linux/PPT_clean/in_vitro/Bea"
path_domains_bea = f"{path_bea}/DepoScope_predictions.bea.domains.0709.fasta"

bea_dico_seq = {record.description : record.seq for record in SeqIO.parse(f"{path_domains_bea}", "fasta") if len(record.seq) >0}
bea_set_records = [record for record in SeqIO.parse(f"{path_domains_bea}", "fasta") if len(record.seq) > 0]

for record in tqdm(bea_set_records) :
    winner = get_winner(record, path_tmp)
    if winner != "No hits" :
        hit = int(winner.split("_")[-1])
        results = dico_pred_correct_name.get(winner, {})
        a = (record.description.split(",")[0] , winner, results)
    else :
        results = "Null"
    a = (record.description.split(",")[0] , winner, results)
    bea_winners.append(a)
    
    
# ***************************************************************************
# Towndsend inferences : 
towndsend_winners = []
path_towndsend = "/media/concha-eloko/Linux/PPT_clean/in_vitro/Townsed"
path_domains_towndsend = f"{path_towndsend}/DepoScope_predictions.Townsed.domains.0909.fasta"

towndsend_dico_seq = {record.description : record.seq for record in SeqIO.parse(f"{path_domains_towndsend}", "fasta") if len(record.seq) >0}
towndsend_set_records = [record for record in SeqIO.parse(f"{path_domains_towndsend}", "fasta") if len(record.seq) > 0]

for record in tqdm(towndsend_set_records) :
    winner = get_winner(record, path_tmp)
    if winner != "No hits" :
        hit = int(winner.split("_")[-1])
        results = dico_pred_correct_name.get(winner, {})
        a = (record.description.split(",")[0] , winner, results)
    else :
        results = "Null"
    a = (record.description.split(",")[0] , winner, results)
    towndsend_winners.append(a)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:01<00:00, 24.91it/s]


In [20]:
TropiLR_results = ferriol_winners + towndsend_winners + bea_winners


In [21]:
with open(f"{path_benchmark}/TropiLogReg.matrices.tsv" , "w") as outfile :
    for prot in TropiLR_results :
        prot_name = prot[0].split("_A")[0]
        if prot[1] == "No hits" :
            outfile.write(f"{prot_name}\tNo_hits\n")
        elif prot[2] == {} :
            outfile.write(f"{prot_name}\tNo_associations\n")
        else :
            try :
                hits = [f"{kltype}:{round(score,3)}" for kltype, score in prot[2].items()]
                outfile.write(f"{prot_name}\t")
                outfile.write(" ; ".join(hits))
                outfile.write("\n")
            except Exception as e :
                print(prot, e)

***
### Work on experimentally validated depolymerases :

In [22]:
# ***************************************************************************
# exp_validated inferences : 
exp_validated_winners = []
path_seq = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

dico_seq = {record.description : record.seq for record in SeqIO.parse(f"{path_seq}/Others_all.dpos_domains.multi.fasta", "fasta") if len(record.seq) >0}
exp_validated_set_records = [record for record in SeqIO.parse(f"{path_seq}/Others_all.dpos_domains.multi.fasta", "fasta") if len(record.seq) > 0]


for record in tqdm(exp_validated_set_records) :
    winner = get_winner(record, path_tmp)
    if winner != "No hits" :
        hit = int(winner.split("_")[-1])
        results = dico_pred_correct_name.get(winner, {})
        a = (record.description.split(",")[0] , winner, results)
    else :
        results = "Null"
    a = (record.description.split(",")[0] , winner, results)
    exp_validated_winners.append(a)
    


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 [00:02<00:00, 23.31it/s]


In [23]:
with open(f"{path_benchmark}/TropiLogReg.exp_val_depolymerase.tsv" , "w") as outfile :
    for prot in exp_validated_winners :
        prot_name = prot[0].split("_A")[0]
        if prot[1] == "No hits" :
            outfile.write(f"{prot_name}\tNo_hits\n")
        elif prot[2] == {} :
            outfile.write(f"{prot_name}\tNo_associations\n")
        else :
            try :
                hits = [f"{kltype}:{round(score,3)}" for kltype, score in prot[2].items()]
                outfile.write(f"{prot_name}\t")
                outfile.write(" ; ".join(hits))
                outfile.write("\n")
            except Exception as e :
                print(prot, e)